In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random

import seaborn as sns
from sklearn.metrics import pairwise_distances
from joblib import Parallel, delayed

from graspologic.embed import AdjacencySpectralEmbed as ASE
from graspologic.embed import LaplacianSpectralEmbed as LSE
from graspologic.simulations import rdpg
from graspologic.simulations import mmsbm
from graspologic.plot import pairplot
from graspologic.plot import heatmap

from tqdm import tqdm
from time import time
from scipy.spatial import distance

import numpy as np
from scipy.spatial import distance

import combining_representations.combining_representations as cr

In [2]:
# Copyright (c) Microsoft Corporation and contributors.
# Licensed under the MIT License.

import warnings
from typing import Any, Callable, Optional, Union

import numpy as np
from sklearn.utils import check_array, check_scalar

from graspologic.types import Dict, List, Tuple
from graspologic.utils import cartesian_product, symmetrize
# from ..utils import cartesian_product, symmetrize


def _n_to_labels(n: np.ndarray) -> np.ndarray:
    n_cumsum = n.cumsum()
    labels = np.zeros(n.sum(), dtype=np.int64)
    for i in range(1, len(n)):
        labels[n_cumsum[i - 1] : n_cumsum[i]] = i
    return labels


def sample_edges(
    P: np.ndarray, directed: bool = False, loops: bool = False
) -> np.ndarray:
 
    if type(P) is not np.ndarray:
        raise TypeError("P must be numpy.ndarray")
    if len(P.shape) != 2:
        raise ValueError("P must have dimension 2 (n_vertices, n_dimensions)")
    if P.shape[0] != P.shape[1]:
        raise ValueError("P must be a square matrix")
    if not directed:
        # can cut down on sampling by ~half
        triu_inds = np.triu_indices(P.shape[0])
        samples = np.random.binomial(1, P[triu_inds])
        A = np.zeros_like(P)
        A[triu_inds] = samples
        A = symmetrize(A, method="triu")
    else:
        A = np.random.binomial(1, P)

    if loops:
        return A
    else:
        return A - np.diag(np.diag(A))


def er_np(
    n: int,
    p: float,
    directed: bool = False,
    loops: bool = False,
    wt: Union[int, np.ndarray, List[int]] = 1,
    wtargs: Optional[Dict[str, Any]] = None,
    dc: Optional[Union[Callable, np.ndarray]] = None,
    dc_kws: Dict[str, Any] = {},
) -> np.ndarray:
   
    if isinstance(dc, (list, np.ndarray)) and all(callable(f) for f in dc):
        raise TypeError("dc is not of type function or array-like of scalars")
    if not np.issubdtype(type(n), np.integer):
        raise TypeError("n is not of type int.")
    if not np.issubdtype(type(p), np.floating):
        raise TypeError("p is not of type float.")
    if type(loops) is not bool:
        raise TypeError("loops is not of type bool.")
    if type(directed) is not bool:
        raise TypeError("directed is not of type bool.")
    n_sbm = np.array([n])
    p_sbm = np.array([[p]])
    g = sbm(n_sbm, p_sbm, directed, loops, wt, wtargs, dc, dc_kws)
    return g  # type: ignore



def er_nm(
    n: int,
    m: int,
    directed: bool = False,
    loops: bool = False,
    wt: Union[int, np.ndarray, List[int]] = 1,
    wtargs: Optional[Dict[str, Any]] = None,
) -> np.ndarray:
    
    if not np.issubdtype(type(m), np.integer):
        raise TypeError("m is not of type int.")
    elif m <= 0:
        msg = "m must be > 0."
        raise ValueError(msg)
    if not np.issubdtype(type(n), np.integer):
        raise TypeError("n is not of type int.")
    elif n <= 0:
        msg = "n must be > 0."
        raise ValueError(msg)
    if type(directed) is not bool:
        raise TypeError("directed is not of type bool.")
    if type(loops) is not bool:
        raise TypeError("loops is not of type bool.")

    # check weight function
    if not np.issubdtype(type(wt), np.integer):
        if not callable(wt):
            raise TypeError("You have not passed a function for wt.")

    # compute max number of edges to sample
    if loops:
        if directed:
            max_edges = n**2
            msg = "n^2"
        else:
            max_edges = n * (n + 1) // 2
            msg = "n(n+1)/2"
    else:
        if directed:
            max_edges = n * (n - 1)
            msg = "n(n-1)"
        else:
            max_edges = n * (n - 1) // 2
            msg = "n(n-1)/2"
    if m > max_edges:
        msg = "You have passed a number of edges, {}, exceeding {}, {}."
        msg = msg.format(m, msg, max_edges)
        raise ValueError(msg)

    A = np.zeros((n, n))
    # check if directedness is desired
    if directed:
        if loops:
            # use all of the indices
            idx = np.where(np.logical_not(A))
        else:
            # use only the off-diagonal indices
            idx = np.where(~np.eye(n, dtype=bool))
    else:
        # use upper-triangle indices, and ignore diagonal according
        # to loops argument
        idx = np.triu_indices(n, k=int(loops is False))

    # get idx in 1d coordinates by ravelling
    triu = np.ravel_multi_index(idx, A.shape)
    # choose M of them
    triu = np.random.choice(triu, size=m, replace=False)
    # unravel back
    triu = np.unravel_index(triu, A.shape)
    # check weight function
    if callable(wt):
        wt = wt(size=m, **wtargs)
    A[triu] = wt

    if not directed:
        A = symmetrize(A, method="triu")

    return A



def sbm(
    n: Union[np.ndarray, List[int]],
    p: np.ndarray,
    directed: bool = False,
    loops: bool = False,
    wt: Union[int, np.ndarray, List[int]] = 1,
    wtargs: Optional[Union[np.ndarray, Dict[str, Any]]] = None,
    dc: Optional[Union[Callable, np.ndarray]] = None,
    dc_kws: Union[Dict[str, Any], List[Dict[str, Any]], np.ndarray] = {},
    return_labels: bool = False,
) -> Union[np.ndarray, Tuple[np.ndarray, np.ndarray]]:
   
    # Check n
    if not isinstance(n, (list, np.ndarray)):
        msg = "n must be a list or np.array, not {}.".format(type(n))
        raise TypeError(msg)
    else:
        n = np.array(n)
        if not np.issubdtype(n.dtype, np.integer):
            msg = "There are non-integer elements in n"
            raise ValueError(msg)

    # Check p
    if not isinstance(p, (list, np.ndarray)):
        msg = "p must be a list or np.array, not {}.".format(type(p))
        raise TypeError(msg)
    else:
        p = np.array(p)
        if not np.issubdtype(p.dtype, np.number):
            msg = "There are non-numeric elements in p"
            raise ValueError(msg)
        elif p.shape != (n.size, n.size):
            msg = "p must have shape len(n) x len(n), not {}".format(p.shape)
            raise ValueError(msg)
        elif np.any(p < 0) or np.any(p > 1):
            msg = "Values in p must be in between 0 and 1."
            raise ValueError(msg)

    # Check wt and wtargs
    if not np.issubdtype(type(wt), np.number) and not callable(wt):
        if not isinstance(wt, (list, np.ndarray)):
            msg = "wt must be a numeric, list, or np.array, not {}".format(type(wt))
            raise TypeError(msg)
        if not isinstance(wtargs, (list, np.ndarray)):
            msg = "wtargs must be a numeric, list, or np.array, not {}".format(
                type(wtargs)
            )
            raise TypeError(msg)

        wt = np.array(wt, dtype=object)
        wtargs = np.array(wtargs, dtype=object)
        # if not number, check dimensions
        if wt.shape != (n.size, n.size):
            msg = "wt must have size len(n) x len(n), not {}".format(wt.shape)
            raise ValueError(msg)
        if wtargs.shape != (n.size, n.size):
            msg = "wtargs must have size len(n) x len(n), not {}".format(wtargs.shape)
            raise ValueError(msg)
        # check if each element is a function
        for element in wt.ravel():
            if not callable(element):
                msg = "{} is not a callable function.".format(element)
                raise TypeError(msg)
    else:
        wt = np.full(p.shape, wt, dtype=object)
        wtargs = np.full(p.shape, wtargs, dtype=object)

    # Check directed
    if not directed:
        if np.any(p != p.T):
            raise ValueError("Specified undirected, but P is directed.")
        if np.any(wt != wt.T):
            raise ValueError("Specified undirected, but Wt is directed.")
        if np.any(wtargs != wtargs.T):
            raise ValueError("Specified undirected, but Wtargs is directed.")

    K = len(n)  # the number of communities
    counter = 0
    # get a list of community indices
    cmties = []
    for i in range(0, K):
        cmties.append(range(counter, counter + n[i]))
        counter += n[i]

    # Check degree-corrected input parameters
    if callable(dc):
        # Check that the paramters are a dict
        if not isinstance(dc_kws, dict):
            msg = "dc_kws must be of type dict not{}".format(type(dc_kws))
            raise TypeError(msg)
        # Create the probability matrix for each vertex
        dcProbs = np.array([dc(**dc_kws) for _ in range(0, sum(n))], dtype="float")
        for indices in cmties:
            dcProbs[indices] /= sum(dcProbs[indices])
    elif isinstance(dc, (list, np.ndarray)) and np.issubdtype(
        np.array(dc).dtype, np.number
    ):
        dcProbs = np.array(dc, dtype=float)
        # Check size and element types
        if not np.issubdtype(dcProbs.dtype, np.number):
            msg = "There are non-numeric elements in dc, {}".format(dcProbs.dtype)
            raise ValueError(msg)
        elif dcProbs.shape != (sum(n),):
            msg = "dc must have size equal to the number of"
            msg += " vertices {0}, not {1}".format(sum(n), dcProbs.shape)
            raise ValueError(msg)
        elif np.any(dcProbs < 0):
            msg = "Values in dc cannot be negative."
            raise ValueError(msg)
        # Check that probabilities sum to 1 in each block
        for i in range(0, K):
            if not np.isclose(sum(dcProbs[cmties[i]]), 1, atol=1.0e-8):
                msg = "Block {} probabilities should sum to 1, normalizing...".format(i)
                warnings.warn(msg, UserWarning)
                dcProbs[cmties[i]] /= sum(dcProbs[cmties[i]])
    elif isinstance(dc, (list, np.ndarray)) and all(callable(f) for f in dc):
        dcFuncs = np.array(dc)
        if dcFuncs.shape != (len(n),):
            msg = "dc must have size equal to the number of blocks {0}, not {1}".format(
                len(n), dcFuncs.shape
            )
            raise ValueError(msg)
        # Check that the parameters type, length, and type
        if not isinstance(dc_kws, (list, np.ndarray)):
            # Allows for nonspecification of default parameters for all functions
            if dc_kws == {}:
                dc_kws = [{} for _ in range(0, len(n))]
            else:
                msg = "dc_kws must be of type list or np.ndarray, not {}".format(
                    type(dc_kws)
                )
                raise TypeError(msg)
        elif not len(dc_kws) == len(n):
            msg = "dc_kws must have size equal to"
            msg += " the number of blocks {0}, not {1}".format(len(n), len(dc_kws))
            raise ValueError(msg)
        elif not all(type(kw) == dict for kw in dc_kws):
            msg = "dc_kws elements must all be of type dict"
            raise TypeError(msg)
        # Create the probability matrix for each vertex
        dcProbs = np.array(
            [
                dcFunc(**kws)
                for dcFunc, kws, size in zip(dcFuncs, dc_kws, n)
                for _ in range(0, size)
            ],
            dtype="float",
        )
        # dcProbs = dcProbs.astype(float)
        for indices in cmties:
            dcProbs[indices] /= sum(dcProbs[indices])
            # dcProbs[indices] = dcProbs / dcProbs[indices].sum()
    elif dc is not None:
        msg = "dc must be a function or a list or np.array of numbers or callable"
        msg += " functions, not {}".format(type(dc))
        raise ValueError(msg)

    # End Checks, begin simulation
    A = np.zeros((sum(n), sum(n)))

    for i in range(0, K):
        if directed:
            jrange = range(0, K)
        else:
            jrange = range(i, K)
        for j in jrange:
            block_wt = wt[i, j]
            block_wtargs = wtargs[i, j]
            block_p = p[i, j]
            # identify submatrix for community i, j
            # cartesian product to identify edges for community i,j pair
            cprod = cartesian_product(cmties[i], cmties[j])  # type: ignore
            # get idx in 1d coordinates by ravelling
            triu = np.ravel_multi_index((cprod[:, 0], cprod[:, 1]), A.shape)
            pchoice = np.random.uniform(size=len(triu))
            if dc is not None:
                # (v1,v2) connected with probability p*k_i*k_j*dcP[v1]*dcP[v2]
                num_edges = sum(pchoice < block_p)
                edge_dist = dcProbs[cprod[:, 0]] * dcProbs[cprod[:, 1]]
                # If n_edges greater than support of dc distribution, pick fewer edges
                if num_edges > sum(edge_dist > 0):
                    msg = "More edges sampled than nonzero pairwise dc entries."
                    msg += " Picking fewer edges"
                    warnings.warn(msg, UserWarning)
                    num_edges = sum(edge_dist > 0)
                triu = np.random.choice(
                    triu, size=num_edges, replace=False, p=edge_dist
                )
            else:
                # connected with probability p
                triu = triu[pchoice < block_p]
            if type(block_wt) is not int:
                block_wt = block_wt(size=len(triu), **block_wtargs)
            triu = np.unravel_index(triu, A.shape)
            A[triu] = block_wt

    if not loops:
        A = A - np.diag(np.diag(A))
    if not directed:
        A = symmetrize(A, method="triu")
    if return_labels:
        labels = _n_to_labels(n)
        return A, labels
    return A



def rdpg(
    X: np.ndarray,
    Y: Optional[np.ndarray] = None,
    rescale: bool = False,
    directed: bool = False,
    loops: bool = False,
    wt: Optional[Union[int, float, Callable]] = 1,
    wtargs: Optional[Dict[str, Any]] = None,
) -> np.ndarray:
    
    P = p_from_latent(X, Y, rescale=rescale, loops=loops)
    A = sample_edges(P, directed=directed, loops=loops)

    # check weight function
    if (not np.issubdtype(type(wt), np.integer)) and (
        not np.issubdtype(type(wt), np.floating)
    ):
        if not callable(wt):
            raise TypeError("You have not passed a function for wt.")

    if callable(wt):
        if wtargs is None:
            wtargs = dict()
        wts = wt(size=(np.count_nonzero(A)), **wtargs)
        A[A > 0] = wts
    else:
        A *= wt  # type: ignore
    return A



def p_from_latent(
    X: np.ndarray,
    Y: Optional[np.ndarray] = None,
    rescale: bool = False,
    loops: bool = True,
) -> np.ndarray:
   
    if Y is None:
        Y = X
    if type(X) is not np.ndarray or type(Y) is not np.ndarray:
        raise TypeError("Latent positions must be numpy.ndarray")
    if X.ndim != 2 or Y.ndim != 2:
        raise ValueError(
            "Latent positions must have dimension 2 (n_vertices, n_dimensions)"
        )
    if X.shape != Y.shape:
        raise ValueError("Dimensions of latent positions X and Y must be the same")
    P = X @ Y.T
    # should this be before or after the rescaling, could give diff answers
    if not loops:
        P = P - np.diag(np.diag(P))
    if rescale:
        if P.min() < 0:
            P = P - P.min()
        if P.max() > 1:
            P = P / P.max()
    else:
        P[P < 0] = 0
        P[P > 1] = 1
    return P


def mmsbm(
    n: int,
    p: np.ndarray,
    alpha: Optional[np.ndarray] = None,
    rng: Optional[np.random.Generator] = None,
    directed: bool = False,
    loops: bool = False,
    return_labels: bool = False,
) -> Union[np.ndarray, Tuple[np.ndarray, np.ndarray]]:
   
    check_scalar(x=n, name="n", target_type=int, min_val=1)

    p = check_array(p, ensure_2d=True)
    nx, ny = p.shape
    if nx != ny:
        msg = "p must be a square matrix, not {}".format(p.shape)
        raise ValueError(msg)
    if not np.issubdtype(p.dtype, np.number):
        msg = "There are non-numeric elements in p"
        raise ValueError(msg)
    if np.any(p < 0) or np.any(p > 1):
        msg = "Values in p must be in between 0 and 1."
        raise ValueError(msg)

    alpha_checked: np.ndarray
    if alpha is None:
        raise ValueError("alpha must not be None")
    else:
        alpha_checked = alpha
        alpha_checked = check_array(
            alpha_checked, ensure_2d=False, ensure_min_features=1
        )
        if not np.issubdtype(alpha_checked.dtype, np.number):
            msg = "There are non-numeric elements in alpha"
            raise ValueError(msg)
        if np.any(alpha_checked <= 0):
            msg = "Alpha entries must be > 0."
            raise ValueError(msg)
        if alpha_checked.shape != (len(p),):
            msg = "alpha must be a list or np.array of shape {c}, not {w}.".format(
                c=(len(p),), w=alpha_checked.shape
            )
            raise ValueError(msg)

            
    if rng == None:
        rng = np.random.default_rng()
    elif not isinstance(rng, np.random.Generator):
        msg = "rng must be <class 'numpy.random.Generator'> not {}.".format(type(rng))
        raise TypeError(msg)

    if type(loops) is not bool:
        raise TypeError("loops is not of type bool.")
    if type(directed) is not bool:
        raise TypeError("directed is not of type bool.")
    if type(return_labels) is not bool:
        raise TypeError("return_labels is not of type bool.")

    if not directed:
        if np.any(p != p.T):
            raise ValueError("Specified undirected, but P is directed.")

    # Naming convention follows paper listed in references.
    mm_vectors = rng.dirichlet(alpha_checked, n)

    mm_vectors = np.array(sorted(mm_vectors, key=lambda x: np.argmax(x)))

    # labels:(n,n) matrix with all membership indicators for initiators and receivers
    # instead of storing the indicator vector, argmax is directly computed
    # check docstrings for more info.
    labels = np.apply_along_axis(
        lambda p_vector: np.argmax(
            rng.multinomial(n=1, pvals=p_vector, size=n), axis=1
        ),
        axis=1,
        arr=mm_vectors,
    )

    P = p[(labels, labels.T)]

    A = sample_edges(P, directed=directed, loops=loops)

    if not loops:
        np.fill_diagonal(labels, -1)

    if return_labels:
        return (A, labels)

    return A, mm_vectors

In [3]:
def precision_calc(inds,S_star,k):
    count = 0
    for ind in inds[:k]:
        if ind in S_star:
            count += 1
    # precision at k
    prec = count / k
    return prec 

In [4]:
def get_dists_S(xhat, vstar, S, invert=False):
    n,d = xhat.shape
    dists_S = np.ones(n)

    
    if len(S) > 0:
        dists_S[np.array(S)] = np.linalg.norm(xhat[vstar] - xhat[S], axis=1)
        non_S = np.array([i for i in range(n) if i not in S and i!=vstar])

        for ind in non_S:
            temp_to_S = np.linalg.norm(xhat[ind] - xhat[S], axis=1)
            argmin = S[np.argmin(temp_to_S)]
            dists_S[ind] = np.linalg.norm(xhat[ind] - xhat[argmin])**2 + np.linalg.norm(xhat[argmin] - xhat[vstar])**2
    
    if invert:
        dists_S = 1 / dists_S
        
    dists_S[vstar] = 0
        
    return dists_S

In [5]:
def plot_current_state(xhat, vstar, S_star, S, T):
    fig, ax = plt.subplots(1,1)
    colors = sns.color_palette("Set1",n_colors=4)
    
    ax.scatter(xhat[:, 0], xhat[:, 1], color='k', alpha=0.1)
    
    ax.scatter(xhat[vstar, 0], xhat[vstar, 1], color='r', marker='*', s=50)
    ax.scatter(xhat[S_star, 0], xhat[S_star, 1], color='y')
    ax.scatter(xhat[S, 0], xhat[S, 1], color='g')
    
    if len(T) > 0:
        ax.scatter(xhat[T, 0], xhat[T, 1], color='k')
    
    return fig, ax

In [1]:
from graspologic.embed import AdjacencySpectralEmbed as ASE

np.random.seed(3)

rank_iters = 10 # nr of times you go through user_in_the_loop
# Nr. of vertices
n = 1000
# Nr. of blocks
K = 2
# P
p = [[0.7,0.2],[0.2,0.05]]
# sbm.alpha
sbm_alpha = [0.1]*2

S_star_size = 20
initial_S_size=3

n_mc=1
n_labelings=5
n_labels_per=10
k = 50

precisions = np.zeros((n_mc, n_labelings+1, 2)) 

for i in range(n_mc): 
    # note: +1 because we have a base case before the user loop
    # Generate instance of Mixed membership SBM
    G = mmsbm(n,p,sbm_alpha)
    
    # Grab membership vectors
    pi_vecs = G[1]

    # Randomly select a node to be vstar
    vstar = np.random.choice(n, 1)[0]
    
    # Calculate "distance" from vstar to other vertices says their block memberships
    dists_to_vstar = np.linalg.norm(pi_vecs - pi_vecs[vstar], axis=1)
        
    # Define S_star to be the S_star_size closest nodes to vstar
    S_star = np.argsort(dists_to_vstar)[1: 1 + S_star_size]
    
    # Define T_star to be everything else
    T_star = [i for i in range(n) if i not in S_star and i != vstar]
    
    print('S star:', S_star)

    xhat = ASE(n_components=2).fit_transform(G[0])
    
    for j in range(n_labelings+1):
        if j == 0:
            # Initial S set
            S = list(np.random.choice(S_star, initial_S_size))

            # Initial T set (empty)
            T = []

        else:
            n_labeled = 0
            c=0
            while n_labeled < n_labels_per:
                if ranks[c] not in np.concatenate([S_and_T, [vstar]]):
                    if ranks[c] in S_star:
                        S = np.concatenate([S, [ranks[c]]])
                    else:
                        T = np.concatenate([T, [ranks[c]]])
                    n_labeled+=1                
                c+=1
                
        print('voi:', vstar)
        print('S:', S)
        print('T:', T)
        
        S_and_T = np.concatenate([S, T]) 
        S = np.array(S).astype(int)
        T = np.array(T).astype(int)
             
            
        # get S and T distances
        dists_S = get_dists_S(xhat, vstar, S)
        dists_T = get_dists_S(xhat, vstar, T, invert=True)
        dists_eucl = np.linalg.norm(xhat[vstar] - xhat, axis=1)

        # create distance matrix
        dist_matrices = np.vstack([dists_S, dists_T, dists_eucl])
        dist_matrices = dist_matrices[np.newaxis].transpose((0,2,1))

        # ilp
        
#         if len(T) > 0:
        alphahat_ilp = cr.multiple_pairs(dist_matrices, {vstar: S}, {vstar: T}, variable_num_tol=None)
        dist_ilp=np.average(dist_matrices[0], weights=alphahat_ilp, axis=1)
        print(alphahat_ilp)
#         else:
#             dist_ilp = dists_S
#             print("just using dists_S")
        
        
        ranks = np.argsort(dist_ilp)
    
        # Remove S and T indices        
        precisions[i, j, 0] = precision_calc(np.argsort(dist_ilp)[1:], S_star, k)
        precisions[i, j, 1] = precision_calc(np.argsort(np.linalg.norm(xhat[vstar] - xhat, axis=1))[1:], S_star, k)
        
        plot_current_state(xhat, vstar, S_star, S, T)
        print()


NameError: name 'np' is not defined

In [7]:
np.argsort(np.linalg.norm(xhat[vstar] - xhat, axis=1))[1:]

array([195, 442, 208, 157,  59, 242, 230, 256, 465,  36,  42, 396, 137,
       388,  51, 104, 382, 309, 266, 145, 240, 427,  37,   8,  22, 115,
        38, 303, 164, 129,  92,  32,  27, 336, 260, 249, 209, 225, 461,
       363,  75,  65, 339, 264, 400, 191, 121, 251, 141, 245,  77, 475,
       275,  28, 128,  95, 474, 448, 323, 346, 193, 295, 361, 341, 322,
       238,  53, 478, 288, 135, 413, 466, 492, 213, 130, 456, 278, 397,
       298, 132, 360, 185, 493, 247,  64, 459, 327, 273,  19,  52, 122,
       182, 227,  68, 317, 228, 146, 237, 441,  48, 270, 179, 271, 326,
       453,   0, 406, 390, 488, 252, 119, 482, 480,  39, 405, 312, 241,
        71, 375, 150, 216, 236, 431, 490,  13,   5,  35, 162,  90, 171,
       138,  16, 356, 332, 420, 444, 108, 325, 365,  33, 451, 310, 387,
       173, 229,  12,  91, 111, 411,  98, 123, 378,  67, 268, 117, 359,
       314, 331, 114, 467,  23, 159,  43,  93, 370, 165, 170, 376, 384,
       460, 476, 235, 419, 269,  76, 307, 352, 197, 282, 358, 42

In [7]:
precisions[0]

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])